In [1]:
# import numpy as np
import pandas as pd
import requests
from datetime import datetime, timedelta

In [2]:
def binder_url(org, repo):
    return f'https://notebooks.gesis.org/binder/v2/gh/{org}/{repo}/master'

In [3]:
def ts_to_dt(ts):
    return datetime.utcfromtimestamp(ts)

# The most popular repositories

In [4]:
def query(time_range):
    query = 'binderhub_launch_time_seconds_count{}[{}]'
    query_selectors = "{status='success'}"
    query = query.format(query_selectors, time_range)
    print(query)
    resp = requests.get('https://notebooks.gesis.org/prometheus/api/v1/query', params={'query': query})
    data = resp.json()['data']['result']
    return data

In [5]:
def process_data(data, time_range_beginning):
    d = {'name': [], 'org': [], 'provider': [], 'launches': [], 'repo_url': [], 'binder_url': []}
    for container in data:
        repo_url = container['metric']['repo']
        provider, org, repo = repo_url.replace('https://', '').rsplit('/', 2)
        
        # calculate number of launches for each repo and each binder container/deployment
        values = [int(ii[1]) for ii in container['values']]
        first_value_ts = container['values'][0][0]
        first_value_dt = datetime.utcfromtimestamp(first_value_ts)
        # prometheus scrapes data each minute, so ignore seconds while comparision
        if first_value_dt.replace(second=0, microsecond=0) > time_range_beginning.replace(second=0, microsecond=0):
            # this container is created after beginning of time range
            # NOTE first value in container can be > 1 if there are simultaneous launches
            # first_value = values[0]
            # assert first_value == 1, f'{org}/{repo}---{first_value}---{first_value_dt}---{time_range_beginning}'
            # print(repo, first_value_dt, time_range_beginning, first_value)
            launches = max(values)
        else:
            # this container is created before beginning of time range
            launches = max(values) - min(values)
                
        # print(repo_url, launches, container['metric']['status'], container['metric']['retries'])
        if repo in d['name']:
            # same repo can have status success with different retries values
            i = d['name'].index(repo)
            d['launches'][i] += launches
        else:
            d['launches'].append(launches)
            
            d['name'].append(repo)
            d['org'].append(org)
            d['provider'].append(provider)
            d['repo_url'].append(repo_url)
            d['binder_url'].append(binder_url(org, repo))
    return d

In [6]:
def makedf(time_range, time_delta):
    # prometheus api returns values for time_range-1m
    # so decreases time_delta 1 min too
    time_delta -= timedelta(minutes=1)
    time_range_beginning = datetime.utcnow() - time_delta
    data = query(f'{time_range}')
    data = process_data(data, time_range_beginning)
    df = pd.DataFrame(data)
    # df = df.drop_duplicates(['name'])
    df = df.groupby(['name', 'org', 'provider','repo_url', 'binder_url']).sum().reset_index().sort_values('launches', ascending=False)
    # df['log_launches'] = df['launches'].apply(np.log)
    df = df.style.format({'repo_url':lambda x: f'<a target="_blank" href="{x}">repo url</a>', 
                 'binder_url': lambda x: f'<a target="_blank" href="{x}">binder url</a>'})
    return df

## Most popular repositories in last hour

In [7]:
hour = 1
time_range = f'{hour}h'
time_delta = timedelta(hours=hour)
df = makedf(time_range, time_delta)
display(df)
"Total number of launches: " + str(sum(df.data['launches'])) + " in "+ str(time_range)

binderhub_launch_time_seconds_count{status='success'}[1h]


,name,org,provider,repo_url,binder_url,launches
4,binder-stats,gesiscss,github.com,repo url,binder url,2
6,ligo-binder,minrk,github.com,repo url,binder url,1
0,BIGSSS,JuKo007,github.com,repo url,binder url,0
1,PythonDataScienceHandbook,gesiscss,github.com,repo url,binder url,0
2,RStan-Binder,arnim,github.com,repo url,binder url,0
3,bdaca,damian0604,github.com,repo url,binder url,0
5,gesis-meta-analysis-2018,berndweiss,github.com,repo url,binder url,0
7,ptm,gesiscss,github.com,repo url,binder url,0
8,stmdemo,arnim,github.com,repo url,binder url,0
9,vtna_frontend,gesiscss,github.com,repo url,binder url,0


'Total number of launches: 3 in 1h'

## Most popular repositories in the last day

In [8]:
day = 1
time_range = f'{day}d'
time_delta = timedelta(days=day)
df = makedf(time_range, time_delta)
display(df)
"Total number of launches: " + str(sum(df.data['launches'])) + " in "+ str(time_range)

binderhub_launch_time_seconds_count{status='success'}[1d]


,name,org,provider,repo_url,binder_url,launches
4,binder-stats,gesiscss,github.com,repo url,binder url,3
10,workshop_girls_day,gesiscss,github.com,repo url,binder url,2
1,PythonDataScienceHandbook,gesiscss,github.com,repo url,binder url,1
6,ligo-binder,minrk,github.com,repo url,binder url,1
7,ptm,gesiscss,github.com,repo url,binder url,1
0,BIGSSS,JuKo007,github.com,repo url,binder url,0
2,RStan-Binder,arnim,github.com,repo url,binder url,0
3,bdaca,damian0604,github.com,repo url,binder url,0
5,gesis-meta-analysis-2018,berndweiss,github.com,repo url,binder url,0
8,stmdemo,arnim,github.com,repo url,binder url,0


'Total number of launches: 8 in 1d'

## Most popular repositories in the last 30 days 

In [9]:
day = 30
time_range = f'{day}d'
time_delta = timedelta(days=day)
df = makedf(time_range, time_delta)
display(df)
"Total number of launches: " + str(sum(df.data['launches'])) + " in "+ str(time_range)

binderhub_launch_time_seconds_count{status='success'}[30d]


,name,org,provider,repo_url,binder_url,launches
48,workshop_girls_day,gesiscss,github.com,repo url,binder url,152
4,PythonDataScienceHandbook,gesiscss,github.com,repo url,binder url,105
8,binder-stats,gesiscss,github.com,repo url,binder url,78
36,ptm,gesiscss,github.com,repo url,binder url,50
17,flow,gesiscss,github.com,repo url,binder url,34
32,ligo-binder,minrk,github.com,repo url,binder url,26
37,pydata-networkx,mriduls,github.com,repo url,binder url,25
5,RStan-Binder,arnim,github.com,repo url,binder url,13
45,stmdemo,arnim,github.com,repo url,binder url,9
41,requirements,binder-examples,github.com,repo url,binder url,9


'Total number of launches: 589 in 30d'

## Most popular repositories in the last 60 days 

In [ ]:
day = 60
time_range = f'{day}d'
time_delta = timedelta(days=day)
df = makedf(time_range, time_delta)
display(df)
"Total number of launches: " + str(sum(df.data['launches'])) + " in "+ str(time_range)

binderhub_launch_time_seconds_count{status='success'}[60d]
